<a href="https://colab.research.google.com/github/opedrohtx/Translogistica/blob/main/translogistica_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Instalando  a biblioteca Gspread**

In [ ]:
#Instalando a Biblioteca Gspread (Responsável pela conexão com o google Planilhas)
!pip install --upgrade gspread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2


##**IMPORT**

In [ ]:
import gspread #importa a biblioteca gspread para a conexão com o Google Planilhas
from tabulate import tabulate #importa a biblioteca tabulate para formatação de tabelas
from google.auth import default #importa a biblioteca google.auth para autenticação no Google
from google.colab import auth #importa a biblioteca google.colab para autenticação no Google Colab
from google.colab import output

auth.authenticate_user() #autentica o usuário no Google Colab
creds, _ = default() #pega as credenciais do usuário de forma padrão

usuario = gspread.authorize(creds) #Variavel usuario recebe a autorição com a autenticação

#abrindo planilha para utilizar no código
spreadsheet = usuario.open('translogistica.sys') #variavel responsavel por abrir a planilha 'sys_cadastro'
cad_viagem = spreadsheet.get_worksheet(0) #variavel responsavel por pegar a primeira aba da planilha
cad_moto = spreadsheet.get_worksheet(1) #variavel responsavel por pegar a segunda aba da planilha
cods = spreadsheet.get_worksheet(2) #variavel responsavel por pegar a terceira aba da planilha

import time
tipo_combustivel = {
    "Diesel" : float(1 * 2.40),
    "Gasolina" : float(1 * 2.95),
    "Alcool" : float(1 * 1.90)
}

from datetime import datetime
data_atual = datetime.today()
data_formatada = data_atual.strftime("%d/%m/%Y")

##**ÁREA DE DADOS**

CADASTRO DE VIAGEM INICIAL

In [ ]:
def cadastro_inicial():
  dados = cad_viagem.get_all_values()
  if len(dados) == 0:
    codigo = int(cods.cell(2,1).value)
  else:
    codigo = int(cods.cell(2,1).value) + 1
    from datetime import datetime
    data_atual = datetime.today()
    data_formatada = data_atual.strftime("%d/%m/%Y")

  data = data_formatada
  hora_viagem = input("Digite a hora de começo da viagem: ")
  tipo_veiculo = input("Digite o tipo de veículo: ")
  tipo_combustivel = input("Digite o tipo de combustível: ")
  placa_veiculo = input("Digite a placa do veículo: ")
  nome = input("Digite o nome do motorista: ")
  odometro_ini = input("Digite o odômetro inicial: ")
  destino = input("Digite o destino: ")

  nova_linha = [codigo, data, hora_viagem, tipo_veiculo, tipo_combustivel, placa_veiculo, nome, odometro_ini, destino]

  cad_viagem.append_row(nova_linha)
  cods.update_cell(2,1, codigo)
  nova_linha1 = [tipo_veiculo, placa_veiculo, nome, tipo_combustivel,]
  cad_moto.append_row(nova_linha1)
  print("Registro adicionado com sucesso!")

  print("O seu codigo de viagem e", codigo)


CADASTRO DE VIAGEM FINAL

In [ ]:
def cadastro_final():
  codigo = input("Digite a codigo de cadastro: ")
  celula = cad_viagem.find(codigo)

  if celula:
    coluna = celula.col
    linha = celula.row

  odometro_inicial = cad_viagem.cell(linha, 8).value
  hora_viagem = cad_viagem.cell(linha, 3).value.replace(':','.')
  if odometro_inicial is None:
    odometro_inicial = 0
  odometro_final = input("Digite o odometro final da viagem: ")
  quilometragem_percorrida = int(odometro_final) - int(odometro_inicial)

  tipo_combustivell = input("Digite o combustível (Diesel, Gasolina, Alcool): ")
  preco_combustivel = tipo_combustivel.get(tipo_combustivell)
  if preco_combustivel is None:
    print("Tipo de combustível invalido.")
    return

  valor_gasto = int(quilometragem_percorrida) * preco_combustivel


  hora_chegada = input("Digite a hora de chegada: ").replace(':','.')
  tempo_total = float(hora_chegada) - float(hora_viagem)




  cad_viagem.update_cell(linha, 12, valor_gasto)
  cad_viagem.update_cell(linha, 10, odometro_final)
  cad_viagem.update_cell(linha, 11, quilometragem_percorrida)
  cad_viagem.update_cell(linha, 13, hora_chegada)
  cad_viagem.update_cell(linha, 14, tempo_total)
  print("Registro atualizado com sucesso!")

LISTAR REGISTRO

In [ ]:
def listar_registros():
  dados = cad_viagem.get_all_values()
  tabela = tabulate(dados, headers=[], tablefmt="rounded_grid", stralign="center")
  print(tabela)


##**MENU**

In [ ]:
while True:

  print("MENU")
  print("1 - Cadastro inicial")
  print("2 - Cadastro final")
  print("3 - Listar")
  print("4 - Sair")
  opcao = float(input("Digite a opção desejada: "))

#Função para cadastrar viagem
  if opcao == 1:
    cadastro_inicial()
    input("Pressione qualquer tecla para continuar...")
    time.sleep(3)
    output.clear()

#Função para finalizar viagem
  elif opcao == 2:
    cadastro_final()
    input("Pressione qualquer tecla para continuar...")
    output.clear()


#Função para listar viagens
  elif opcao == 3:
    listar_registros()
    input("Pressione qualquer tecla para continuar...")
    output.clear()

#Função para sair do programa
  elif opcao == 4:
    print("Saindo...")
    break

  else:
    print("Opção inválida!")
    input("Pressione qualquer tecla para continuar...")

MENU
1 - Cadastro inicial
2 - Cadastro final
3 - Listar
4 - Sair
Digite a opção desejada: 4
Saindo...
